In [247]:
import os
import json 

output_path = "valentine_output_valentine_original"
schemapile_responses_path = "responses_starcoder_schemapile_valentine.json"
valentine_ground_truth_path = "valentine_ground_truth_valentine"
output_path_combined = "output/JaccardLevenMatcher/"
hybrid = True

files = {}
for filename in os.listdir(output_path):
    if os.path.isfile(os.path.join(output_path, filename)):
        files[filename] = json.load(open(os.path.join(output_path, filename)))

In [248]:
import json
schemapile_valentine = json.load(open(schemapile_responses_path))
schemapile_valentine_clean = {}
for dataset in schemapile_valentine:
    dataset_name = dataset[dataset.rfind("/")+1:]
    schemapile_valentine_clean[dataset_name] = schemapile_valentine[dataset]

In [249]:
def find_schemapile_numbers(name):
    data = None
    found = 0
    for dataset in schemapile_valentine_clean:
        if dataset.startswith(name):
            found += 1
            data = schemapile_valentine_clean[dataset]
    if found > 1:
        print("more than 1 candidate found, taking last one: "+name)
    return data

In [250]:
import ast
from data_loader.golden_standard_loader import GoldenStandardLoader
from utils.utils import get_project_root, create_folder
import metrics.metrics as module_metric

metric_names=["precision",
"recall",
"f1_score",
"precision_at_n_percent",
"recall_at_sizeof_ground_truth"]

metric_fns = [getattr(module_metric, met) for met in metric_names]

In [251]:
for file in files:
    name = file.split("__")[0]
    files[file]["schemapile"] = find_schemapile_numbers(name)
    matches = files[file]["matches"]
    
    if files[file]["schemapile"] is None:
        print("skipped, because not file found: "+file)
        continue

    match_found = None
    for match in matches:  
        match_rating = matches[match]
        source_col = ast.literal_eval(match)[0][1].lower()
        target_col = ast.literal_eval(match)[1][1].lower()
        
        if source_col == files[file]["schemapile"]["column"] and target_col == files[file]["schemapile"]["referencedColumn"]:
            match_found = match
            if hybrid:
                matches[match] = 1.1
    
    if not hybrid:
        matches = {}
        if match_found:
            matches[match_found] = 1
        else:
            match = f'(("{files[file]["schemapile"]["table"]}", "{files[file]["schemapile"]["column"]}"), ("{files[file]["schemapile"]["referencedTable"]}", "{files[file]["schemapile"]["referencedColumn"]}"))'
            matches[match] = 1.1
        files[file]["matches"] = matches
        matches = dict(sorted(matches.items(), key=lambda item: -item[1]))
            
    if match_found:
        matches = dict(sorted(matches.items(), key=lambda item: -item[1]))
    
    files[file]["matches"] = matches

    golden_standard = GoldenStandardLoader(f"{valentine_ground_truth_path}/{name}/{name}_mapping.json")
    matches_structured = {ast.literal_eval(k): v for k,v in files[file]["matches"].items()}

    if len(golden_standard.expected_matches) != 1:
        continue

    updated_metrics = {}
    
    for metric in metric_fns:
        if metric.__name__ != "precision_at_n_percent":
            if metric.__name__ in ['precision', 'recall', 'f1_score']:
                updated_metrics[metric.__name__] = metric(matches_structured, golden_standard, True)
            else:
                updated_metrics[metric.__name__] = metric(matches_structured, golden_standard)
        else:
            for n in range(10,100,10):
                updated_metrics[metric.__name__.replace('_n_', '_' + str(n) + '_')] = metric(matches_structured, golden_standard, n)

    files[file]["metrics"] = updated_metrics
    json.dump(files[file],open(output_path_combined+file,"w+"))